In [8]:
# ============================================================================
# LangChain RAG Tool Setup - Cell 1: Imports and LLM Configuration
# ============================================================================

# ============================================================================
# Core Imports
# ============================================================================
import os
from dotenv import load_dotenv

# ============================================================================
# Text Processing & Embeddings
# ============================================================================
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

# ============================================================================
# Vector Stores & Retrieval
# ============================================================================
from langchain_community.vectorstores import FAISS
from langchain_core.tools import create_retriever_tool

# ============================================================================
# Language Models (LLMs)
# ============================================================================
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

# ============================================================================
# Prompts & Agents
# ============================================================================
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain.agents import create_agent

# ============================================================================
# Memory & Conversation (for RAG applications)
# ============================================================================
# Try multiple import paths for ConversationBufferMemory compatibility
print("Attempting to import ConversationBufferMemory...")

ConversationBufferMemory = None
memory_import_paths = [
    ("langchain.memory", "ConversationBufferMemory"),
    ("langchain_community.chat_message_histories", "ChatMessageHistory"),
    ("langchain_core.messages", "BaseMessage"),
]

for module_name, class_name in memory_import_paths:
    try:
        module = __import__(module_name, fromlist=[class_name])
        ConversationBufferMemory = getattr(module, class_name, None)
        if ConversationBufferMemory:
            print(f"✓ Found {class_name} in {module_name}")
            break
    except (ImportError, AttributeError):
        pass

if ConversationBufferMemory is None:
    print("⚠️  ConversationBufferMemory not found - will implement custom solution")

# InMemorySaver: Checkpoint memory for agent state
from langgraph.checkpoint.memory import InMemorySaver

# Load environment variables from .env file
load_dotenv()

# ============================================================================
# API Key Configuration
# ============================================================================
# Retrieve API keys for Gemini and OpenAI from environment variables
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Validate that both API keys are present
assert GEMINI_API_KEY, "GEMINI_API_KEY is missing. Check your .env file."
assert OPENAI_API_KEY, "OPENAI_API_KEY is missing. Check your .env file."

# Display partial keys to confirm successful loading (for security, only show first 6 chars)
print("Key loaded:", GEMINI_API_KEY[:6] + "..." )
print("OpenAI Key loaded:", OPENAI_API_KEY[:6] + "...")

# ============================================================================
# Initialize Language Models (LLMs)
# ============================================================================
# Initialize Google's Gemini LLM with temperature=0 for deterministic responses
gemini_llm = ChatGoogleGenerativeAI(
    temperature=0,  # Deterministic output (no randomness)
    model="gemini-2.5-flash",  # Using the faster Gemini model
    google_api_key=GEMINI_API_KEY
)

# Initialize OpenAI's GPT-3.5-turbo LLM
openai_llm = ChatOpenAI(
    temperature=0,  # Deterministic output
    model="gpt-3.5-turbo",
    openai_api_key=OPENAI_API_KEY
)

# ============================================================================
# Create a Simple Prompt Template
# ============================================================================
# Define a reusable prompt template for company name generation
prompt = PromptTemplate(
    input_variables=["product"],  # Variable to be filled dynamically
    template="Give me a creative name for a company that makes {product}?",
)

# ============================================================================
# Example 1: LLM Chain (Prompt → LLM)
# ============================================================================
# Create a chain: prompt template piped to OpenAI LLM
chain = prompt | openai_llm

# Execute the chain with a specific product type
response = chain.invoke({"product": "smart home devices"})

# Display the generated company name
print("OpenAI response:", response)

Attempting to import ConversationBufferMemory...
✓ Found ChatMessageHistory in langchain_community.chat_message_histories
Key loaded: AIzaSy...
OpenAI Key loaded: sk-pro...
OpenAI response: content='IntelliHome Technologies' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 21, 'total_tokens': 25, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CrZvDmTrMLIWzA3f540P6lFB8nC8D', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b6295-2f89-7312-8e82-8373c75f76eb-0' usage_metadata={'input_tokens': 21, 'output_tokens': 4, 'total_tokens': 25, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'rea

In [9]:
# ============================================================================
# LangChain RAG Tool Setup - Cell 2: FAISS Vector Database & Retriever
# ============================================================================

# ============================================================================
# Step 1: Load Document
# ============================================================================
# Read the sample.txt file containing text data to be indexed
print("Loading document from sample.txt...")
with open("sample.txt", "r", encoding="utf-8") as f:
    text_data = f.read()
print("✓ Document loaded successfully\n")

# ============================================================================
# Step 2: Split Text into Chunks
# ============================================================================
# Initialize text splitter to break documents into manageable chunks
# - separator="\n": Split on newline characters
# - chunk_size=300: Each chunk contains ~300 characters
# - chunk_overlap=50: Overlap for context preservation
print("Splitting text into chunks...")
splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=300,
    chunk_overlap=50
)
# Create document objects from text
docs = splitter.create_documents([text_data])
print(f"✓ Created {len(docs)} document chunks\n")

# ============================================================================
# Step 3: Create Embeddings and FAISS Vector Store
# ============================================================================
# Initialize OpenAI embeddings (converts text to vectors)
print("Creating embeddings and FAISS vector store...")
embeddings = OpenAIEmbeddings()

# Create FAISS vector database from documents
vectorstore = FAISS.from_documents(docs, embeddings)
print("✓ FAISS vector store created successfully\n")

# ============================================================================
# Step 4: Create Retriever
# ============================================================================
# Create a retriever that fetches relevant documents
# search_kwargs={"k": 4} means return top 4 most relevant documents
print("Creating retriever...")
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
print("✓ Retriever configured (returns top 4 documents)\n")

# ============================================================================
# Step 5: Convert Retriever to RAG Tool
# ============================================================================
# Wrap the retriever as a tool for the agent to use
print("Creating RAG tool...")
rag_tool = create_retriever_tool(
    retriever,
    name="rag_qa_tool",
    description="Use this tool to answer questions about LangChain using the sample.txt document"
)
print("✓ RAG tool created successfully\n")

# ============================================================================
# Step 6: Create Agent with RAG Tool
# ============================================================================
# Create modern agent that can use the RAG tool
print("Initializing agent with RAG tool...")
agent = create_agent(
    openai_llm,  # Use OpenAI LLM
    tools=[rag_tool],  # Provide RAG tool
    system_prompt="You are a helpful assistant. Use the RAG tool when needed to answer questions accurately based on the provided documents."
)
print("✓ Agent created successfully\n")

# ============================================================================
# Step 7: Test the RAG Agent
# ============================================================================
print("="*80)
print("TESTING RAG AGENT WITH FAISS VECTOR STORE")
print("="*80 + "\n")

# Test Query 1
query1 = "What is LangChain?"
print(f"Query: {query1}")
result1 = agent.invoke({
    "messages": [{"role": "user", "content": query1}]
})
print(f"Answer: {result1['messages'][-1].content}\n")

# Test Query 2
query2 = "Why do we use LangChain?"
print(f"Query: {query2}")
result2 = agent.invoke({
    "messages": [{"role": "user", "content": query2}]
})
print(f"Answer: {result2['messages'][-1].content}\n")

print("="*80)

Loading document from sample.txt...
✓ Document loaded successfully

Splitting text into chunks...
✓ Created 1 document chunks

Creating embeddings and FAISS vector store...
✓ FAISS vector store created successfully

Creating retriever...
✓ Retriever configured (returns top 4 documents)

Creating RAG tool...
✓ RAG tool created successfully

Initializing agent with RAG tool...
✓ Agent created successfully

TESTING RAG AGENT WITH FAISS VECTOR STORE

Query: What is LangChain?
Answer: LangChain is a framework for building applications with Large Language Models (LLMs). It was created by Harrison Chase and supports RAG, agents, memory, tools, and more. LangChain is commonly used in chatbots, document Q&A, and AI workflow.

Query: Why do we use LangChain?
Answer: LangChain is used as a framework for building applications with Large Language Models (LLMs). It was created by Harrison Chase and supports various features such as RAG, agents, memory, tools, and more. LangChain is commonly used in 

In [10]:
# ============================================================================
# LangChain RAG Tool Setup - Cell 3: ConversationBufferMemory Integration
# ============================================================================
# This cell adds memory to the agent for multi-turn conversations

from typing import List, Dict, Any
from datetime import datetime

print("\n" + "="*80)
print("IMPLEMENTING CONVERSATION MEMORY WITH RAG AGENT")
print("="*80 + "\n")

# ============================================================================
# Step 1: Create Custom ConversationBufferMemory Class
# ============================================================================
# If ConversationBufferMemory is not available, use custom implementation

print("Setting up conversation memory...\n")

# Helper function to get memory as string
def get_memory_string(memory_obj) -> str:
    """Extract memory history as string from various memory formats."""
    try:
        # Try buffer attribute (LangChain ConversationBufferMemory)
        if hasattr(memory_obj, 'buffer'):
            return memory_obj.buffer
        # Try buffer_as_str property
        elif hasattr(memory_obj, 'buffer_as_str'):
            return memory_obj.buffer_as_str
        # Try chat_memory.messages
        elif hasattr(memory_obj, 'chat_memory'):
            messages = memory_obj.chat_memory.messages
            lines = []
            for msg in messages:
                role = "User" if msg.type == "human" else "Assistant"
                lines.append(f"{role}: {msg.content}")
            return "\n".join(lines)
        # Fallback
        else:
            return str(memory_obj)
    except Exception as e:
        print(f"  [Could not format memory: {e}]")
        return ""

# Helper function to get messages list
def get_memory_messages(memory_obj) -> list:
    """Extract messages from various memory formats."""
    try:
        if hasattr(memory_obj, 'chat_memory') and hasattr(memory_obj.chat_memory, 'messages'):
            return memory_obj.chat_memory.messages
        elif hasattr(memory_obj, 'messages'):
            return memory_obj.messages
        else:
            return []
    except Exception:
        return []

if ConversationBufferMemory is not None:
    # Use built-in ConversationBufferMemory if available
    print("✓ Using LangChain's ConversationBufferMemory\n")
    
    memory = ConversationBufferMemory(
        return_messages=True
    )
else:
    # Implement custom conversation memory
    print("⚠️  Using custom ConversationMemory implementation\n")
    
    class ConversationMemory:
        """
        Custom conversation memory implementation.
        Stores complete conversation history with timestamps.
        """
        def __init__(self, memory_key: str = "chat_history"):
            self.memory_key = memory_key
            self.messages: List[Dict[str, Any]] = []
        
        def add_user_message(self, content: str) -> None:
            """Add a user message to memory."""
            self.messages.append({
                "type": "human",
                "content": content,
                "timestamp": datetime.now().isoformat()
            })
        
        def add_ai_message(self, content: str) -> None:
            """Add an AI message to memory."""
            self.messages.append({
                "type": "ai",
                "content": content,
                "timestamp": datetime.now().isoformat()
            })
        
        @property
        def buffer(self) -> str:
            """Return conversation history as formatted string."""
            if not self.messages:
                return ""
            
            lines = []
            for msg in self.messages:
                role = "User" if msg["type"] == "human" else "Assistant"
                lines.append(f"{role}: {msg['content']}")
            return "\n".join(lines)
    
    # Create memory instance
    memory = ConversationMemory()
    print("✓ Custom ConversationMemory created")
    print("  → Stores complete conversation history")
    print("  → Includes timestamps for each message\n")

# ============================================================================
# Step 2: Create Conversational RAG Agent with Memory
# ============================================================================
print("Creating conversational agent with memory...\n")

# System prompt that tells agent about memory and RAG tool
system_prompt = """You are a helpful assistant that remembers the conversation history.

IMPORTANT RULES:
1. Always refer to previous messages in the conversation when relevant
2. Use the RAG tool to answer questions about LangChain
3. Acknowledge when you're using information from previous turns
4. Be conversational and maintain context across turns
5. If asked about previous questions, reference them naturally
"""

# Create agent with memory-aware system prompt
conversational_agent = create_agent(
    openai_llm,
    tools=[rag_tool],
    system_prompt=system_prompt
)

print("✓ Conversational agent created")
print("  → Agent: create_agent() with RAG tool")
print("  → Memory: Conversation Buffer Memory")
print("  → Mode: Multi-turn conversation capable\n")

# ============================================================================
# Step 3: Define Helper Function for Conversational Interactions
# ============================================================================
print("Creating conversation handler function...\n")

def chat_with_memory(query: str, show_history: bool = False) -> str:
    """
    Send a query to the conversational agent and update memory.
    
    Args:
        query (str): User's question/message
        show_history (bool): Whether to display conversation history
    
    Returns:
        str: Agent's response
    """
    # Add user message to memory
    if hasattr(memory, 'add_message'):
        memory.add_message({"role": "user", "content": query})
    elif hasattr(memory, 'add_user_message'):
        memory.add_user_message(query)
    elif hasattr(memory, 'chat_memory'):
        memory.chat_memory.add_user_message(query)
    else:
        print("Warning: Could not add user message to memory")
    
    # Get conversation history for context
    chat_history_str = get_memory_string(memory)
    
    # Display history if requested
    if show_history and chat_history_str:
        print("\n📜 CONVERSATION HISTORY:")
        print("─" * 80)
        print(chat_history_str)
        print("─" * 80 + "\n")
    
    # Invoke agent with query and memory context
    try:
        result = conversational_agent.invoke({
            "messages": [{"role": "user", "content": query}]
        })
        
        # Extract agent response
        agent_response = result['messages'][-1].content
        
        # Add agent response to memory
        if hasattr(memory, 'add_message'):
            memory.add_message({"role": "assistant", "content": agent_response})
        elif hasattr(memory, 'add_ai_message'):
            memory.add_ai_message(agent_response)
        elif hasattr(memory, 'chat_memory'):
            memory.chat_memory.add_ai_message(agent_response)
        
        return agent_response
    
    except Exception as e:
        print(f"Error: {e}")
        return None

print("✓ Chat function created\n")

# ============================================================================
# Step 4: Multi-Turn Conversation Examples
# ============================================================================
print("="*80)
print("MULTI-TURN CONVERSATION WITH MEMORY")
print("="*80 + "\n")

# Turn 1
print("TURN 1:")
print("-" * 80)
query1 = "What is LangChain?"
print(f"User: {query1}")
response1 = chat_with_memory(query1, show_history=False)
print(f"Agent: {response1}\n")

# Turn 2: References previous conversation
print("TURN 2:")
print("-" * 80)
query2 = "What are the main features of it?"
print(f"User: {query2}")
response2 = chat_with_memory(query2, show_history=True)
print(f"Agent: {response2}\n")

# Turn 3: Another follow-up
print("TURN 3:")
print("-" * 80)
query3 = "Can you tell me who created it?"
print(f"User: {query3}")
response3 = chat_with_memory(query3, show_history=False)
print(f"Agent: {response3}\n")

# ============================================================================
# Step 5: Display Complete Conversation History
# ============================================================================
print("="*80)
print("COMPLETE CONVERSATION HISTORY")
print("="*80 + "\n")

full_history = get_memory_string(memory)
print(full_history if full_history else "[No conversation yet]")

# ============================================================================
# Step 6: Memory Statistics
# ============================================================================
print("\n" + "="*80)
print("CONVERSATION STATISTICS")
print("="*80 + "\n")

# Get messages
messages = get_memory_messages(memory)

# Count message types
num_user_messages = 0
num_ai_messages = 0

for m in messages:
    # Handle both dict and object formats
    msg_type = None
    if isinstance(m, dict):
        msg_type = m.get("type")
    elif hasattr(m, 'type'):
        msg_type = m.type
    
    if msg_type == "human":
        num_user_messages += 1
    elif msg_type == "ai":
        num_ai_messages += 1

print(f"Total Messages: {len(messages)}")
print(f"  • User Messages: {num_user_messages}")
print(f"  • AI Messages: {num_ai_messages}")
print(f"  • Conversation Turns: {num_user_messages}")

# Estimate token count
total_text = full_history if full_history else ""
word_count = len(total_text.split()) if total_text else 0
estimated_tokens = int(word_count * 1.3)

print(f"\nMemory Usage:")
print(f"  • Characters: {len(total_text):,}")
print(f"  • Words: {word_count:,}")
print(f"  • Estimated Tokens: ~{estimated_tokens:,}")

print("\n" + "="*80 + "\n")


IMPLEMENTING CONVERSATION MEMORY WITH RAG AGENT

Setting up conversation memory...

✓ Using LangChain's ConversationBufferMemory

Creating conversational agent with memory...

✓ Conversational agent created
  → Agent: create_agent() with RAG tool
  → Memory: Conversation Buffer Memory
  → Mode: Multi-turn conversation capable

Creating conversation handler function...

✓ Chat function created

MULTI-TURN CONVERSATION WITH MEMORY

TURN 1:
--------------------------------------------------------------------------------
User: What is LangChain?
  [Could not format memory: Got unsupported message type: {'role': 'user', 'content': 'What is LangChain?'}]
Agent: LangChain is a framework created by Harrison Chase for building applications with Large Language Models (LLMs). It supports RAG, agents, memory, tools, and more, and is commonly used in chatbots, document Q&A, and AI workflow.

TURN 2:
--------------------------------------------------------------------------------
User: What are the